# Neural Collaborative Filter
In this notebook, we will develop a neural collaborative folder that aims to recommend a movie to a user based on their previous watching patterns. We will use two architectures, one an item-based recommender and the other a user-based recommender. An item-based recommender will take the genres of movies and find similar movies based on the user's preferences. A user-based recommender will find other similar users based on this like their age, occupation and gender and then recommend movies that other similar people have enjoyed.

In [1]:
pip install keras-tuner

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense, Dropout
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import keras_tuner as kt
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import load_model

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense, Dropout
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

We will now import our test and train data sets. The test data set takes the latest 20% of ratings from each user. So this data is not random, but aims to simulate what would happen in real life. There is another train/test split available which just has the test data as the latest 20% of ratings. We will compare the difference in performance of the models for using each data split later in the project. However, for now we will only use the split that takes the last 20% of raing from each individual user.

In [3]:
train_df = pd.read_csv('../Katherine W/dataSets/user_train_df.csv')
test_df = pd.read_csv('../Katherine W/dataSets/user_test_df.csv')


we now prepare the input data for the first model

In [4]:
train_df.index = range(1, len(train_df) + 1)
test_df.index = range(1, len(test_df) + 1)

In [5]:
num_users = 943
num_items = 1682
num_genres = 19
num_occupations = train_df['Occupation'].nunique()
num_genders = train_df['Gender'].nunique()


In [6]:
# Convert all specified columns to numeric, setting errors='coerce' to convert non-numeric values to NaN
genre_columns = ['Fantasy', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western','Film-Noir','Unknown','Action','Adeventure','Animation','Childrens','Comedy','Crime','Documentary','Drama']

train_df[genre_columns] = train_df[genre_columns].apply(pd.to_numeric, errors='coerce')

# Drop rows where any of the specified columns contain NaN values
train_df = train_df.dropna(subset=genre_columns)


test_df[genre_columns] = test_df[genre_columns].apply(pd.to_numeric, errors='coerce')
test_df = test_df.dropna(subset=genre_columns)

In [7]:
# Encoders
occupation_encoder = LabelEncoder()
gender_encoder = LabelEncoder()

In [8]:
# Prepare Input Features
genre_features = genre_columns
train_genre_input = train_df[genre_features].values
test_genre_input = test_df[genre_features].values
train_df['Occupation'] = occupation_encoder.fit_transform(train_df['Occupation'])
test_df['Occupation'] = occupation_encoder.transform(test_df['Occupation'])
train_df['Gender'] = gender_encoder.fit_transform(train_df['Gender'])
test_df['Gender'] = gender_encoder.fit_transform(test_df['Gender'])

Now to build the model

In [9]:

# Model Architecture
# Inputs
user_input = Input(shape=(1,), name='User_Input')
item_input = Input(shape=(1,), name='Item_Input')
genre_input = Input(shape=(num_genres,), name='Genre_Input')
age_input = Input(shape=(1,), name='Age_Input')
occupation_input = Input(shape=(1,), name='Occupation_Input')
gender_input = Input(shape=(1,), name='Gender Input')

In [10]:

# Embedding layers for user and item
user_embedding = Embedding(num_users+1, 50, name='User_Embedding')(user_input)
item_embedding = Embedding(num_items+1, 50, name='Item_Embedding')(item_input)
age_embedding = Embedding(110, 50, name='Age_Embedding')(age_input)
occupation_embedding = Embedding(num_occupations, 50, name='Occupation_Embedding')(occupation_input)
gender_embedding = Embedding(num_genders, 50, name='Gender_Embedding')(gender_input)

In [11]:
# Flatten embeddings
user_vec = Flatten()(user_embedding)
item_vec = Flatten()(item_embedding)
age_vec = Flatten()(age_embedding)
occupation_vec = Flatten()(occupation_embedding)
gender_vec = Flatten()(gender_embedding)

In [12]:
# Concatenate embeddings and genre input
concat = Concatenate()([user_vec, item_vec, genre_input,age_vec,occupation_vec,gender_vec])

In [13]:
# Dense layers
dense1 = Dense(128, activation='relu')(concat)
dropout1 = Dropout(0.3)(dense1)
dense2 = Dense(64, activation='relu')(dropout1)

output = Dense(1, activation='sigmoid')(dense2)

In [14]:
# Model definition
model = Model(inputs=[user_input, item_input, genre_input,age_input,occupation_input,gender_input], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


Prepare the training data. If the rating is above 3, we claim that the user enjoyed the film. We treat this as a binary variable, (1 if the user enjoyed the film and 0 otherwise)

In [15]:
# Preparing training data
train_user_input = train_df['User ID'].values
train_item_input = train_df['Item ID'].values
train_age_input = train_df['Age'].values
train_occupation_input = train_df['Occupation'].values
train_gender_input = train_df['Gender'].values
train_ratings = (train_df['Rating'] > 3).astype(int).values  # Binary rating: 1 if >3, else 0



test_user_input = test_df['User ID'].values
test_item_input = test_df['Item ID'].values
test_age_input = test_df['Age'].values
test_occuption_input = test_df['Occupation'].values
test_gender_input = test_df['Gender'].values
test_ratings = (test_df['Rating'] > 3).astype(int).values


In [16]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ User_Input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Item_Input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Age_Input           │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Occupation_Input    │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Gender Input        │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ User_Embedding      │ (None, 1, 50)     │     47,200 │ User_Input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Item_Embedding      │ (None, 1, 50)     │     84,150 │ Item_Input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Age_Embedding       │ (None, 1, 50)     │      5,500 │ Age_Input[0][0]   │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Occupation_Embeddi… │ (None, 1, 50)     │      1,050 │ Occupation_Input… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Gender_Embedding    │ (None, 1, 50)     │        100 │ Gender            │
│ (Embedding)         │                   │            │ Input[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 50)        │          0 │ User_Embedding[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 50)        │          0 │ Item_Embedding[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Genre_Input         │ (None, 19)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_2 (Flatten) │ (None, 50)        │          0 │ Age_Embedding[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_3 (Flatten) │ (None, 50)        │          0 │ Occupation_Embed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_4 (Flatten) │ (None, 50)        │          0 │ Gender_Embedding… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 269)       │          0 │ flatten[0][0],    │
│ (Concatenate)       │                   │            │ flatten_1[0][0],  │
│                     │                   │            │ Genre_Input[0][0… │
│                     │                   │            │ flatten_2[0][0],  │
│                     │                   │            │ flatten_3[0][0],  │
│                     │                   │            │ flatten_4[0][0] 

 Total params: 180,881 (706.57 KB)

 Trainable params: 180,881 (706.57 KB)

 Non-trainable params: 0 (0.00 B)

Now to train the model

In [17]:
# Training the model
history = model.fit(
    [train_user_input, train_item_input, train_genre_input, train_age_input,train_occupation_input,train_gender_input],
    train_ratings,
    validation_data=([test_user_input, test_item_input, test_genre_input, test_age_input,test_occuption_input,test_gender_input], test_ratings),
    epochs=10,
    batch_size=64
)

Epoch 1/10


c:\Users\44747\anaconda3\envs\GroupAssignment2\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['User_Input', 'Item_Input', 'Genre_Input', 'Age_Input', 'Occupation_Input', 'Gender Input']. Received: the structure of inputs=('*', '*', '*', '*', '*', '*')
  warnings.warn(


499/499 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6365 - loss: 0.6358 - val_accuracy: 0.7422 - val_loss: 0.5191
Epoch 2/10
499/499 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7241 - loss: 0.5455 - val_accuracy: 0.7645 - val_loss: 0.4936
Epoch 3/10
499/499 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7332 - loss: 0.5287 - val_accuracy: 0.7789 - val_loss: 0.4676
Epoch 4/10
499/499 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7501 - loss: 0.5039 - val_accuracy: 0.7906 - val_loss: 0.4436
Epoch 5/10
499/499 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7622 - loss: 0.4859 - val_accuracy: 0.8110 - val_loss: 0.4281
Epoch 6/10
499/499 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7787 - loss: 0.4639 - val_accuracy: 0.8211 - val_loss: 0.4099
Epoch 7/10
499/499 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7856 - loss: 0.4498 - val_accuracy: 0.8351 - val_loss: 0.3781
Epoch 8/10
499/499 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8006 - loss: 0.4262 - val_accuracy: 0.8545 - val_

The model has now been built and trained. Now to test it out. We take a random user, user_id, to see how the model behaves

In [18]:
# Predicting Recommendations for a User
user_id = 212  # Example: User ID for which to recommend
user_movies = np.array(range(num_items))  # All movies
# Find the age corresponding to the user ID in the train_df
user_age = train_df.loc[train_df['User ID'] == user_id, 'Age'].values[0]
user_occupation = train_df.loc[train_df['User ID'] == user_id, 'Occupation'].values[0]
user_gender = train_df.loc[train_df['User ID']== user_id, 'Gender'].values[0]

In [19]:
# Combine the DataFrames vertically (stacking them on top of each other)
combined_df = pd.concat([train_df, test_df], ignore_index=True)

genre_df = combined_df.copy()

genre_df.drop(columns=['timestamp','Age','Gender','Occupation','zip code','Release Date','URL','Movie Title','User ID','Item ID','Rating'],inplace=True)

# Genres for all movies
movie_genres_input = genre_df[:user_movies.shape[0]]

In [20]:
# Ensure all inputs are properly shaped and converted to the correct dtype
user_input_predict = np.full((user_movies.shape[0], 1), user_id, dtype=np.int32)  # Shape: (num_items, 1)
item_input_predict = user_movies.reshape(-1, 1).astype(np.int32)  # Shape: (num_items, 1)
movie_genres_input = movie_genres_input.astype(np.float32)  # Ensure genre input is float32
age_input_predict = np.full((user_movies.shape[0], 1), user_age, dtype=np.int32)  # Shape: (num_items, 1)
occupation_input_predict = np.full((user_movies.shape[0], 1), user_occupation, dtype=np.int32)  # Shape: (num_items, 1)
gender_input_predict = np.full((user_movies.shape[0], 1), user_gender, dtype=np.int32)  # Shape: (num_items, 1)

In [21]:
# Predict scores
predicted_scores = model.predict([np.full(user_movies.shape, user_id), user_movies, movie_genres_input,age_input_predict,occupation_input_predict,gender_input_predict])


21/53 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step  

c:\Users\44747\anaconda3\envs\GroupAssignment2\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['User_Input', 'Item_Input', 'Genre_Input', 'Age_Input', 'Occupation_Input', 'Gender Input']. Received: the structure of inputs=('*', '*', '*', '*', '*', '*')
  warnings.warn(


53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [22]:
# Recommend Top-5 Movies
recommended_movies = np.argsort(-predicted_scores.flatten())[:10]
print("Top 10 recommended movies for User ID", user_id, ":", recommended_movies)

Top 10 recommended movies for User ID 212 : [1616 1467  174 1599 1500 1396 1449  199 1466 1368]


Now we print the details of the recommended movies:

In [23]:
filtered_df = train_df[train_df['Item ID'].isin(recommended_movies)]

filtered_df.drop_duplicates(subset='Item ID', inplace=True)

filtered_df.drop(columns=['User ID','Rating','timestamp','Age','Gender','Occupation','zip code'],inplace=True)
filtered_df

C:\Users\44747\AppData\Local\Temp\ipykernel_39656\992600379.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.drop_duplicates(subset='Item ID', inplace=True)
C:\Users\44747\AppData\Local\Temp\ipykernel_39656\992600379.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.drop(columns=['User ID','Rating','timestamp','Age','Gender','Occupation','zip code'],inplace=True)


,Item ID,Movie Title,Release Date,URL,Unknown,Action,Adeventure,Animation,Childrens,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
30,199,"Bridge on the River Kwai, The (1957)",01-Jan-1957,http://us.imdb.com/M/title-exact?Bridge%20on%2...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
53,174,Raiders of the Lost Ark (1981),01-Jan-1981,http://us.imdb.com/M/title-exact?Raiders%20of%...,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5538,1368,Mina Tannenbaum (1994),01-Jan-1994,http://us.imdb.com/M/title-exact?Mina%20Tannen...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5725,1396,Stonewall (1995),26-Jul-1996,http://us.imdb.com/M/title-exact?Stonewall%20(...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7400,1449,Pather Panchali (1955),22-Mar-1996,http://us.imdb.com/M/title-exact?Pather%20Panc...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7652,1466,Margarets Museum (1995)|01-Jan-1995||http://us...,0,0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7706,1467,"Saint of Fort Washington, The (1993)",01-Jan-1993,http://us.imdb.com/M/title-exact?Saint%20of%20...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9123,1500,Santa with Muscles (1996),08-Nov-1996,http://us.imdb.com/M/title-exact?Santa%20with%...,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16044,1599,Someone Elses America (1995)|10-May-1996||http...,0,0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18380,1616,Desert Winds (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Desert%20Wind...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now we look at the genres for each of these movies:

In [24]:
# Function to get column names with positive entries for each row
def get_positive_columns(row):
    return [col for col in row.index if row[col] == 1]

# Apply the function to each row
positive_columns_per_row = filtered_df.apply(get_positive_columns, axis=1)

# Show the result
for idx, positive_cols in enumerate(positive_columns_per_row):
    print(f"Row {idx} has positive values in columns: {positive_cols}")

Row 0 has positive values in columns: ['Drama', 'War']
Row 1 has positive values in columns: ['Action', 'Adeventure']
Row 2 has positive values in columns: ['Drama']
Row 3 has positive values in columns: ['Drama']
Row 4 has positive values in columns: ['Drama']
Row 5 has positive values in columns: ['Comedy']
Row 6 has positive values in columns: ['Drama']
Row 7 has positive values in columns: ['Comedy']
Row 8 has positive values in columns: ['Comedy']
Row 9 has positive values in columns: ['Drama']


Compare the genres to see if they are similar to the movies in the test data that the user has watched and liked.

In [25]:
true_ratings = test_df.loc[test_df['User ID'] == user_id].sort_values(by=['Rating'], ascending=False).head(5)

true_ratings.drop(columns=['User ID','URL','Rating','timestamp','Age','Gender','Occupation','zip code','Release Date'], inplace = True)

true_ratings

,Item ID,Movie Title,Unknown,Action,Adeventure,Animation,Childrens,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
1267,423,E.T. the Extra-Terrestrial (1982),0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1266,179,"Clockwork Orange, A (1971)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [26]:
# Apply the function to each row
positive_columns_per_row = true_ratings.apply(get_positive_columns, axis=1)

# Show the result
for idx, positive_cols in enumerate(positive_columns_per_row):
    print(f"Row {idx} has positive values in columns: {positive_cols}")

Row 0 has positive values in columns: ['Childrens', 'Drama', 'Fantasy', 'Sci-Fi']
Row 1 has positive values in columns: ['Sci-Fi']


## Look at tuning hyper-paramaters
We now look to optimise our model.

Here we have also added an additional dense layer, the thinking behind this was to capture more complex patterns between the information we had on the user for each review, this did come at the cost of increasing the time to get predictions quite considerably so would advise against re running cells in this file unless you have some time free.

In [27]:
# Assuming df has been prepared with 'UserID', 'ItemID', 'Rating', and genre columns

# HyperModel for the neural network
class CollaborativeFilterHyperModel(kt.HyperModel):
    def build(self, hp):
        # Input dimensions
        num_users = 943
        num_items = 1682
        num_genres = 19
        num_occupations = train_df['Occupation'].nunique()
        num_genders = train_df['Gender'].nunique()
        # Model inputs
        user_input = Input(shape=(1,), name='User_Input')
        item_input = Input(shape=(1,), name='Item_Input')
        genre_input = Input(shape=(num_genres,), name='Genre_Input')
        occupation_input = Input(shape=(1,), name='Occupation_Input')
        gender_input = Input(shape=(1,), name='Gender_Input')

        # Hyperparameters for embeddings and dense layers
        user_embedding_dim = hp.Int('user_embedding_dim', min_value=32, max_value=128, step=32)
        item_embedding_dim = hp.Int('item_embedding_dim', min_value=32, max_value=128, step=32)
        age_embedding_dim = hp.Int('age_embedding_dim', min_value=32, max_value=128, step=32)
        occupation_embedding_dim = hp.Int('occupation_embedding_dim', min_value=32, max_value=128, step=32)
        gender_embedding_dim = hp.Int('gender_embedding_dim', min_value=32, max_value=128, step=32)
        
        # Embedding layers for user and item
        user_embedding = Embedding(num_users+1, user_embedding_dim, name='User_Embedding')(user_input)
        item_embedding = Embedding(num_items+1, item_embedding_dim, name='Item_Embedding')(item_input)
        age_embedding = Embedding(110, age_embedding_dim, name='Age_Embedding')(age_input)
        occupation_embedding = Embedding(num_occupations, occupation_embedding_dim, name='Occupation_Embedding')(occupation_input)
        gender_embedding = Embedding(num_genders, gender_embedding_dim, name='Gender_Embedding')(gender_input)
        
        # Flatten the embeddings
        user_vec = Flatten()(user_embedding)
        item_vec = Flatten()(item_embedding)
        age_vec = Flatten()(age_embedding)
        occupation_vec = Flatten()(occupation_embedding)
        gender_vec = Flatten()(gender_embedding)
        
        # Concatenate embeddings with genre input
        concat = Concatenate()([user_vec, item_vec, genre_input,age_vec,occupation_vec,gender_vec])

        # Dense layers with hyperparameter search
        dense1_units = hp.Int('dense1_units', min_value=64, max_value=512, step=64)
        dense2_units = hp.Int('dense2_units', min_value=32, max_value=256, step=32)
        dense3_units = hp.Int('dense3_units', min_value=16, max_value=128, step=16)
        
        dense1 = Dense(dense1_units, activation='relu')(concat)
        dropout1 = Dropout(hp.Float('dropout1', min_value=0.2, max_value=0.5, step=0.1))(dense1)
        dense2 = Dense(dense2_units, activation='relu')(dropout1)
        dense3 = Dense(dense3_units, activation='relu')(dense2)
        # Output layer
        output = Dense(1, activation='linear')(dense3)  # Rating is continuous, use linear activation

        # Model definition
        model = Model(inputs=[user_input, item_input, genre_input,age_input,occupation_input,gender_input], outputs=output)

        # Compile the model with an optimizer and loss function
        model.compile(optimizer=Adam(), loss='mse', metrics=['mae'])

        return model

# Preparing the training and testing data (just like before)


train_user_input = train_df['User ID'].values
train_item_input = train_df['Item ID'].values
train_genre_input = train_df[genre_columns].values
train_ratings = train_df['Rating'].values
train_age_input = train_df['Age'].values
train_occupation_input = train_df['Occupation'].values
train_gender_input = train_df['Gender'].values

test_user_input = test_df['User ID'].values
test_item_input = test_df['Item ID'].values
test_genre_input = test_df[genre_columns].values
test_ratings = test_df['Rating'].values
test_age_input = test_df['Age'].values
test_occupation_input = test_df['Occupation'].values
test_gender_input = test_df['Gender'].values

# Instantiate the tuner
tuner = kt.RandomSearch(
    CollaborativeFilterHyperModel(),
    objective='val_mae',  # We are optimizing for Mean Absolute Error
    max_trials=5,  # Number of different hyperparameter combinations to try
    executions_per_trial=3,  # Number of executions for each trial
    #directory='"C:/Users/kwhit/OneDrive/Documents/Maths 4th year/Data Science Toolbox 2024/Group-Assignment-2/Katherine W"',  # Directory to save tuning results
    project_name='collab_filter_search_all_features_3dense'  # Project name for Keras Tuner
)

# Search for the best hyperparameters
tuner.search(
    [train_user_input, train_item_input, train_genre_input,train_age_input,train_occupation_input,train_gender_input],
    train_ratings,
    validation_data=([test_user_input, test_item_input, test_genre_input,test_age_input,test_occupation_input,test_gender_input], test_ratings),
    epochs=10,
    batch_size=64
)

# Retrieve the best hyperparameters
best_hp = tuner.get_best_hyperparameters()[0]
print("Best hyperparameters:", best_hp.values)

# Build the model with the best hyperparameters
best_model = tuner.hypermodel.build(best_hp)

# Train the model using the best hyperparameters
history = best_model.fit(
    [train_user_input, train_item_input, train_genre_input,train_age_input,train_occupation_input,train_gender_input],
    train_ratings,
    validation_data=([test_user_input, test_item_input, test_genre_input,test_age_input,test_occupation_input,test_gender_input], test_ratings),
    epochs=10,
    batch_size=64
)

# Evaluate the best model
test_loss, test_mae = best_model.evaluate([test_user_input, test_item_input, test_genre_input,test_age_input,test_occupation_input,test_gender_input], test_ratings)
print(f"Test MAE: {test_mae}")

Trial 5 Complete [00h 01m 27s]
val_mae: 0.545649786790212

Best val_mae So Far: 0.5207668344179789
Total elapsed time: 00h 06m 54s
Best hyperparameters: {'user_embedding_dim': 96, 'item_embedding_dim': 128, 'age_embedding_dim': 32, 'occupation_embedding_dim': 64, 'gender_embedding_dim': 64, 'dense1_units': 448, 'dense2_units': 256, 'dense3_units': 112, 'dropout1': 0.4}
Epoch 1/10
499/499 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 1.7910 - mae: 1.0115 - val_loss: 0.8822 - val_mae: 0.7612
Epoch 2/10
499/499 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.9138 - mae: 0.7589 - val_loss: 0.8163 - val_mae: 0.7329
Epoch 3/10
499/499 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.8527 - mae: 0.7308 - val_loss: 0.7777 - val_mae: 0.7144
Epoch 4/10
499/499 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.7897 - mae: 0.7011 - val_loss: 0.6863 - val_mae: 0.6515
Epoch 5/10
499/499 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.7305 - mae: 0.6738 - val_loss: 0.6292 - val_mae: 0.6294
Epoch 6/10
499/499 ━━━━━━━━━━━━━━━━━━━

In [28]:
# Save the best model to a file
best_model.save('best_item_collab_filter_model.h5')
print("Model saved!")

Model saved!


In [29]:
from tensorflow.keras.losses import MeanSquaredError

# Register the custom metric
mse = MeanSquaredError()

# Load the saved model
item_model = load_model('best_item_collab_filter_model.h5', custom_objects={'mse': mse})
print("Model loaded!")

Model loaded!


In [30]:
item_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ User_Input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Item_Input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Age_Input           │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Occupation_Input    │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Gender_Input        │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ User_Embedding      │ (None, 1, 96)     │     90,624 │ User_Input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Item_Embedding      │ (None, 1, 128)    │    215,424 │ Item_Input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Age_Embedding       │ (None, 1, 32)     │      3,520 │ Age_Input[0][0]   │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Occupation_Embeddi… │ (None, 1, 64)     │      1,344 │ Occupation_Input… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Gender_Embedding    │ (None, 1, 64)     │        128 │ Gender_Input[0][… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_5 (Flatten) │ (None, 96)        │          0 │ User_Embedding[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_6 (Flatten) │ (None, 128)       │          0 │ Item_Embedding[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Genre_Input         │ (None, 19)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_7 (Flatten) │ (None, 32)        │          0 │ Age_Embedding[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_8 (Flatten) │ (None, 64)        │          0 │ Occupation_Embed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_9 (Flatten) │ (None, 64)        │          0 │ Gender_Embedding… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 403)       │          0 │ flatten_5[0][0],  │
│ (Concatenate)       │                   │            │ flatten_6[0][0],  │
│                     │                   │            │ Genre_Input[0][0… │
│                     │                   │            │ flatten_7[0][0],  │
│                     │                   │            │ flatten_8[0][0],  │
│                     │                   │            │ flatten_9[0][0] 

 Total params: 635,875 (2.43 MB)

 Trainable params: 635,873 (2.43 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

Now time to test our new optimised model in a similar way to before:

In [31]:
# Predicting Recommendations for a User
user_id = 76  # Example: User ID for which to recommend
user_movies = np.array(range(num_items))  # All movies

In [32]:
# Combine the DataFrames vertically (stacking them on top of each other)
combined_df = pd.concat([train_df, test_df], ignore_index=True)

genre_df = combined_df.copy()

genre_df.drop(columns=['timestamp','Age','Gender','Occupation','zip code','Release Date','URL','Movie Title','User ID','Item ID','Rating'],inplace=True)

In [33]:
# Genres for all movies
movie_genres_input = genre_df[:user_movies.shape[0]]

In [34]:
# Find the age corresponding to the user ID in the train_df
user_age = train_df.loc[train_df['User ID'] == user_id, 'Age'].values[0]
print(f"The age corresponding to User ID {user_id} is: {user_age}")
# Find the occupation corresponding to the user ID in the train_df
user_occupation = train_df.loc[train_df['User ID'] == user_id, 'Occupation'].values[0]
print(f"The occupation corresponding to User ID {user_id} is: {user_occupation}")
# Find the gender corresponding to the user ID in the train_df
user_gender = train_df.loc[train_df['User ID'] == user_id, 'Gender'].values[0]
print(f"The gender corresponding to User ID {user_id} is: {user_gender}")

The age corresponding to User ID 76 is: 20
The occupation corresponding to User ID 76 is: 18
The gender corresponding to User ID 76 is: 1


In [35]:
# Ensure all inputs are properly shaped and converted to the correct dtype
user_input_predict = np.full((user_movies.shape[0], 1), user_id, dtype=np.int32)  # Shape: (num_items, 1)
item_input_predict = user_movies.reshape(-1, 1).astype(np.int32)  # Shape: (num_items, 1)
movie_genres_input = movie_genres_input.astype(np.float32)  # Ensure genre input is float32
age_input_predict = np.full((user_movies.shape[0], 1), user_age, dtype=np.int32)  # Shape: (num_items, 1)
occupation_input_predict = np.full((user_movies.shape[0], 1), user_occupation, dtype=np.int32)  # Shape: (num_items, 1)
gender_input_predict = np.full((user_movies.shape[0], 1), user_gender, dtype=np.int32)  # Shape: (num_items, 1)

In [36]:
# Predict scores
predicted_scores = item_model.predict([np.full(user_movies.shape, user_id), user_movies, movie_genres_input,age_input_predict,occupation_input_predict,gender_input_predict])


c:\Users\44747\anaconda3\envs\GroupAssignment2\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['User_Input', 'Item_Input', 'Genre_Input', 'Age_Input', 'Occupation_Input', 'Gender_Input']. Received: the structure of inputs=('*', '*', '*', '*', '*', '*')
  warnings.warn(


53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


In [37]:
# Recommend Top-5 Movies
recommended_movies = np.argsort(-predicted_scores.flatten())[:10]
print("Top 10 recommended movies for User ID", user_id, ":", recommended_movies)

Top 10 recommended movies for User ID 76 : [ 868  192 1467 1599 1600  174 1646 1070  853 1525]


In [38]:
filtered_df = train_df[train_df['Item ID'].isin(recommended_movies)]

filtered_df.drop_duplicates(subset='Item ID', inplace=True)
filtered_df.drop(columns=['User ID','Rating','timestamp','Age','Gender','Occupation','zip code'],inplace=True)

filtered_df

C:\Users\44747\AppData\Local\Temp\ipykernel_39656\1878379110.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.drop_duplicates(subset='Item ID', inplace=True)
C:\Users\44747\AppData\Local\Temp\ipykernel_39656\1878379110.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.drop(columns=['User ID','Rating','timestamp','Age','Gender','Occupation','zip code'],inplace=True)


,Item ID,Movie Title,Release Date,URL,Unknown,Action,Adeventure,Animation,Childrens,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
38,192,Raging Bull (1980),01-Jan-1980,http://us.imdb.com/M/title-exact?Raging%20Bull...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53,174,Raiders of the Lost Ark (1981),01-Jan-1981,http://us.imdb.com/M/title-exact?Raiders%20of%...,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
618,868,Hearts and Minds (1996),10-Jan-1997,http://us.imdb.com/M/title-exact?Hearts%20and%...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
695,853,Braindead (1992),01-Jan-1992,http://us.imdb.com/M/title-exact?Braindead%20(...,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1734,1070,Living in Oblivion (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Living%20in%2...,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7706,1467,"Saint of Fort Washington, The (1993)",01-Jan-1993,http://us.imdb.com/M/title-exact?Saint%20of%20...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11932,1525,"Object of My Affection, The (1998)",20-Mar-1998,http://us.imdb.com/Title?Object+of+My+Affectio...,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
16044,1599,Someone Elses America (1995)|10-May-1996||http...,0,0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16109,1600,Guantanamera (1994),16-May-1997,http://us.imdb.com/M/title-exact?Guantanamera%...,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23565,1646,Men With Guns (1997),06-Mar-1998,http://us.imdb.com/Title?Men+with+Guns+(1997/I),0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
# Function to get column names with positive entries for each row
def get_positive_columns(row):
    return [col for col in row.index if row[col] == 1]

# Apply the function to each row
positive_columns_per_row = filtered_df.apply(get_positive_columns, axis=1)

# Show the result
for idx, positive_cols in enumerate(positive_columns_per_row):
    print(f"Row {idx} has positive values in columns: {positive_cols}")

Row 0 has positive values in columns: ['Drama']
Row 1 has positive values in columns: ['Action', 'Adeventure']
Row 2 has positive values in columns: ['Drama']
Row 3 has positive values in columns: ['Comedy', 'Horror']
Row 4 has positive values in columns: ['Comedy']
Row 5 has positive values in columns: ['Drama']
Row 6 has positive values in columns: ['Comedy', 'Romance']
Row 7 has positive values in columns: ['Comedy']
Row 8 has positive values in columns: ['Comedy']
Row 9 has positive values in columns: ['Action', 'Drama']


In [40]:
true_ratings = test_df.loc[test_df['User ID'] == user_id].sort_values(by=['Rating'], ascending=False).head(5)

true_ratings.drop(columns=['User ID','URL','Rating','timestamp','Age','Gender','Occupation','zip code','Release Date'], inplace = True)

true_ratings

,Item ID,Movie Title,Unknown,Action,Adeventure,Animation,Childrens,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
508,582,"Piano, The (1993)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
506,223,Sling Blade (1996),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
507,1048,Shes the One (1996)|23-Aug-1996||http://us.imd...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
# Apply the function to each row
positive_columns_per_row = true_ratings.apply(get_positive_columns, axis=1)

# Show the result
for idx, positive_cols in enumerate(positive_columns_per_row):
    print(f"Row {idx} has positive values in columns: {positive_cols}")

Row 0 has positive values in columns: ['Drama', 'Romance']
Row 1 has positive values in columns: ['Drama', 'Thriller']
Row 2 has positive values in columns: ['Adeventure', 'Horror']


# Look at coverage

In [42]:

# Step 1: Initialize a set to track recommended item IDs
recommended_items = set()

# Step 2: Loop through all users in the dataset
for user_id in train_df['User ID'].unique():
    user_movies = np.array(range(num_items))  # All movies
    user_age = train_df.loc[train_df['User ID'] == user_id, 'Age'].values[0]
    user_occupation = train_df.loc[train_df['User ID'] == user_id, 'Occupation'].values[0]
    user_gender = train_df.loc[train_df['User ID']==user_id, 'Gender'].values[0]
    # Combine the DataFrames vertically (stacking them on top of each other)
    combined_df = pd.concat([train_df, test_df], ignore_index=True)

    genre_df = combined_df.copy()
    genre_df.drop(columns=['timestamp', 'Age', 'Gender', 'Occupation', 'zip code', 
                           'Release Date', 'URL', 'Movie Title', 'User ID', 
                           'Item ID', 'Rating'], inplace=True)
    
    # Genres for all movies
    movie_genres_input = genre_df[:user_movies.shape[0]]
    
    # Ensure all inputs are properly shaped and converted to the correct dtype
    user_input_predict = np.full((user_movies.shape[0],), user_id, dtype=np.int32)  # Shape: (num_items,)
    item_input_predict = user_movies.astype(np.int32)  # Shape: (num_items,)
    movie_genres_input = movie_genres_input.astype(np.float32)  # Ensure genre input is float32
    age_input_predict = np.full((user_movies.shape[0], 1), user_age, dtype=np.int32)  # Shape: (num_items,)
    occupation_input_predict = np.full((user_movies.shape[0], 1), user_occupation, dtype=np.int32)  # Shape: (num_items,)
    gender_input_predict = np.full((user_movies.shape[0],1), user_gender, dtype=np.int32)  # Shape: (num_items,)
    # Predict scores
    predicted_scores = item_model.predict([user_input_predict, item_input_predict, movie_genres_input,age_input_predict,occupation_input_predict,gender_input_predict]).flatten()

    # Convert predictions to NumPy for easier manipulation
    predicted_scores = predicted_scores.flatten()

    # Get top N recommendations (e.g., N=10)
    top_n_indices = np.argsort(predicted_scores)[::-1][:10]  # Get indices of top N scores

    # Add the top recommended items to the set
    recommended_items.update(user_movies[top_n_indices])  # Use user_movies to map indices to actual item IDs

# Step 3: Calculate coverage
coverage = (len(recommended_items) / num_items) * 100
print(f"Coverage: {coverage:.2f}%")


c:\Users\44747\anaconda3\envs\GroupAssignment2\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['User_Input', 'Item_Input', 'Genre_Input', 'Age_Input', 'Occupation_Input', 'Gender_Input']. Received: the structure of inputs=('*', '*', '*', '*', '*', '*')
  warnings.warn(


53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━

In [43]:
len(recommended_items)

194

In [44]:
print(f"Coverage: {coverage:.2f}%")

Coverage: 11.53%


## MSE of model predictions

In [45]:

# Step 1: Initialize lists to store the results
user_ids = []
item_ids = []
actual_ratings = []
predicted_ratings = []

# Step 2: Loop through each user in the dataset
for user_id in combined_df['User ID'].unique():
    # Get all items rated by this user
    user_data = combined_df[combined_df['User ID'] == user_id]
    items_rated_by_user = user_data['Item ID'].values
    actual_ratings_dict = dict(zip(user_data['Item ID'], user_data['Rating']))

    user_age = user_data['Age'].values[0]
    user_occupation = user_data['Occupation'].values[0]
    user_gender = user_data['Gender'].values[0]
    
    # Predict scores for all items
    user_movies = np.array(range(num_items))  # All movies
    user_input = np.full((num_items,), user_id, dtype=np.int32)
    item_input = user_movies.astype(np.int32)
    genre_input = genre_df.values[:num_items]  # Ensure genre_input has the same size as num_items
    age_input = np.full((num_items, 1), user_age, dtype=np.int32)  # Shape: (num_items, 1)
    occupation_input = np.full((num_items, 1), user_occupation, dtype=np.int32)  # Shape: (num_items, 1)
    gender_input = np.full((num_items, 1), user_gender, dtype=np.int32)  # Shape: (num_items, 1)
    # Predict scores
    predicted_scores = item_model.predict([user_input, item_input, genre_input,age_input,occupation_input,gender_input]).flatten()

    # Append data for each movie
    for idx, item_id in enumerate(user_movies):
        user_ids.append(user_id)
        item_ids.append(item_id)
        predicted_ratings.append(predicted_scores[idx])
        # Get actual rating if it exists; otherwise, append NaN
        actual_ratings.append(actual_ratings_dict.get(item_id, np.nan))

# Step 3: Create a DataFrame
results_df = pd.DataFrame({
    'User ID': user_ids,
    'Item ID': item_ids,
    'Actual Rating': actual_ratings,
    'Predicted Rating': predicted_ratings
})

# Display the first few rows of the DataFrame
print(results_df.head())


53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
53/53 ━━━━━━━━━━━━━━━━━━━

In [46]:
results_df = results_df.dropna()
results_df

,User ID,Item ID,Actual Rating,Predicted Rating
1,1,1,5.0,3.305405
2,1,2,3.0,4.114155
3,1,3,4.0,3.526108
4,1,4,3.0,4.614108
5,1,5,3.0,3.661820
...,...,...,...,...
1583790,943,1028,2.0,2.142781
1583806,943,1044,3.0,3.141064
1583809,943,1047,2.0,3.153020
1583829,943,1067,2.0,3.291162


In [47]:
#Calculating the mse:


# Assuming 'results_df' has the columns 'UserID', 'PredictedRating', and 'ActualRating'

# Step 1: Initialize an empty list to store MSE values for each user
mse_per_user = []

# Step 2: Group by UserID and calculate MSE for each user
for user_id, user_data in results_df.groupby('User ID'):
    # Get actual and predicted ratings for this user
    actual_ratings = user_data['Actual Rating']
    predicted_ratings = user_data['Predicted Rating']
    
    # Calculate MSE for this user
    mse = np.mean((actual_ratings - predicted_ratings) ** 2)
    
    # Append the MSE value to the list
    mse_per_user.append({'User ID': user_id, 'MSE': mse})

# Convert the list to a DataFrame for better readability (optional)
mse_df = pd.DataFrame(mse_per_user)

# Show the MSE values per user
mse_df

,User ID,MSE
0,1,1.004106
1,2,0.724907
2,3,0.615834
3,4,0.265770
4,5,0.995202
...,...,...
937,939,0.588445
938,940,0.853700
939,941,0.872423
940,942,0.938401


In [48]:
mean_value = mse_df['MSE'].mean()

print(f"MSE of the model overal is: {mean_value}")

MSE of the model overal is: 0.7570712188346286


## Counting correlated suggestions

In [50]:

# Initialize a counter for the event
event_count = 0

# Iterate over each user in the test_df
for user_id in test_df['User ID'].unique():
    # Get the actual movies that the user has in the test_df (movies they rated)
    user_test_data = test_df[test_df['User ID'] == user_id]
    
    # Filter movies with a rating above 3
    user_test_data = user_test_data[user_test_data['Rating'] > 3]
    
    if user_test_data.empty:
        continue  # Skip the user if no movies have a rating above 3
    
    # Get the movies and their ratings
    test_movies = user_test_data['Item ID'].values
    test_ratings = user_test_data['Rating'].values
    
    # Get the predicted scores for this user (all movies)
    user_movies = np.array(range(num_items))  # All movies
    genre_input = genre_df.values  # Assuming genre_df has the necessary genre features
    user_age = user_test_data['Age'].values[0]
    user_occupation = user_test_data['Occupation'].values[0]
    user_gender = user_test_data['Gender'].values[0]
    # Ensure that user_input is shaped as (num_items, 1), so it's a column vector
    user_input_predict = np.full((user_movies.shape[0], 1), user_id, dtype=np.int32)  # Shape: (num_items, 1)
    item_input_predict = user_movies.reshape(-1, 1).astype(np.int32)  # Shape: (num_items, 1)
    age_input_predict = np.full((user_movies.shape[0], 1), user_age, dtype=np.int32)  # Shape: (num_items, 1)
    occupation_input_predict = np.full((user_movies.shape[0], 1), user_occupation, dtype=np.int32)  # Shape: (num_items, 1)
    gender_input_predict = np.full((user_movies.shape[0], 1), user_gender, dtype=np.int32)  # Shape: (num_items, 1)
    # Ensure genre_input is of shape (num_items, n_genres)
    genre_input_predict = genre_input[:user_movies.shape[0], :].astype(np.int32)  # Shape: (num_items, n_genres)
    # Predict the scores for all items (movies)
    predicted_scores = item_model.predict([user_input_predict, item_input_predict, genre_input_predict,age_input_predict,occupation_input_predict,gender_input_predict]).flatten()
    
    # Get the top 10 recommended movies based on the predicted scores
    top_n_indices = np.argsort(predicted_scores)[::-1][:10]  # Top 10 movies
    
    # Loop through each movie in the test data
    for movie_id in test_movies:
        if movie_id in top_n_indices:
            event_count += 1

# Print the final count of events
print(f"Number of times a movie with a rating above 3 appeared in the top 10 recommendations: {event_count}")


21/53 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step  

c:\Users\44747\anaconda3\envs\GroupAssignment2\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['User_Input', 'Item_Input', 'Genre_Input', 'Age_Input', 'Occupation_Input', 'Gender_Input']. Received: the structure of inputs=('*', '*', '*', '*', '*', '*')
  warnings.warn(


53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
53/53 ━━━━━━━━━━━━━━━━━━━

In [51]:
print('The number of times a recommended movie has appeared in the test set and actually liked (rating above 3) is:',event_count)

The number of times a recommended movie has appeared in the test set and actually liked (rating above 3) is: 128


In [52]:
# Filter the test_df to only include rows where the rating is greater than 3
ratings_above_3 = test_df[test_df['Rating'] > 3]

# Count how many rows meet this condition
count_ratings_above_3 = ratings_above_3.shape[0]

# Print the result
print(f"Number of ratings above 3: {count_ratings_above_3}")
print('Percentage of enjoyed movies that show up in recommendations is:',(event_count/count_ratings_above_3)*100)


Number of ratings above 3: 3012
Percentage of enjoyed movies that show up in recommendations is: 4.249667994687915


## Scaling the model

Recommender systems are likely to have to deal with huge volumes of data, we therefore think about how our model would cope with being scaled to 100000 times its size. Neural networks are known for coping well wih large amounts of data, and actually perform better when there is more data available. As more users join, and with more data being provided the neural netowrk will learn more and more pathways which had previously been quite sparse. Therefore it is expected that as the volume of data increases the neura netwrks performance will also increase. 

On the other hand, with larger volumes of data, the model will take a long time to train. 

## MSE
The MSE for the model is less about 0.9. As the ratings are out of 5, this would mean that on average a predicted rating is out by around 1. Although this isn't particuarly good, as the model ranks the top 10 it can be argued that it is likely that the model will suggest films that the user would enjoy (rating >=3). It is unlikely that the model would suggest a movie that the user would acually rate 2 or less.

## Coverage
The coverage of the model is quite low, at 10.54%, which means that the model favours particular movies over others. This could be because collaborative filters are quite heavily influenced on popularity which can lead to over-fitting. With this in mind, when we look to scale the model by 100,000 times it is likely that the coverage will increase. This is because wih more data, any sparse data will be filled in and so the network can learn patterns about the less popular options. With more data comes more diversity and so with better understanding of the more niche preferences it's hoped that the model will be able to provide more personalised recommendations.

## Matching Recommendations to Test Data
The proportion of recommendations that actually appeared in the test set is incredibly low at only 3.1%, however his does not provide a particuarly good measure for how well the model is performing. Although the users didnt actually watch many of the model's recommendations it does not mean that they wouldn't enjoy them. It's likely that the model recommends movies that they will enjoy more than the ones they actually watched (movies in test set). As the model is scaled up, it isn't particurly likely that this percentage will increase, however as explained this is not necessarily a bad thing.